# Importing Libraries

In [1]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, f1_score
import time
import pickle 

warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2

# Reading Files

In [3]:
crude_oil_prices = pd.read_csv('../InputData/crude_oil_prices.csv')
federal_rates = pd.read_csv('../InputData/effective_federal_funds_rate.csv')
corridor_rates = pd.read_csv('../InputData/egyptian_corridor_interest_rates.csv')
housing_index = pd.read_csv('../InputData/housing_index.csv')
inflation_mom = pd.read_csv('../InputData/inflation_month_on_month.csv')
inflation_yoy = pd.read_csv('../InputData/inflation_year_on_year.csv')
news_data = pd.read_csv('../InputData/news.csv')
stock_prices = pd.read_csv('../InputData/stocks_prices_and_volumes.csv')
vix_indices = pd.read_csv('../InputData/vix_index.csv')
vixeem_indices = pd.read_csv('../InputData/vxeem_index.csv')
gold_prices = pd.read_csv('../InputData/intraday_gold.csv')
gold_prices_target = pd.read_csv('../InputData/target_gold.csv')

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\omarf\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:

import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Load the CSV file
df = pd.read_csv('../InputData/news.csv')

# Initialize VADER sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Function to calculate sentiment using VADER
def get_sentiment(text):
    return sia.polarity_scores(text)['compound']  # Returns the compound score

# Extract sentiment scores along with the date
df['sentiment_score'] = df['summary'].apply(get_sentiment)
sentiment_data = df[['date', 'sentiment_score']]

# Convert date to datetime if needed
sentiment_data['date'] = pd.to_datetime(sentiment_data['date'])

# Print or save the result
sentiment_data.to_csv('sentiment_scores.csv', index=False)


C:\Users\omarf\AppData\Local\Temp\ipykernel_19572\3694712986.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_data['date'] = pd.to_datetime(sentiment_data['date'])


In [4]:
print(news_data.head)

<bound method NDFrame.head of             id source       category                 date  \
0      1281441      d          banks  2020-01-01 00:00:00   
1      1281462      d          banks  2020-01-01 00:00:00   
2      1281542      d          banks  2020-01-01 00:00:00   
3      1281542      d          banks  2020-01-01 00:00:00   
4      3579478      e  announcements  2020-01-02 10:50:00   
...        ...    ...            ...                  ...   
91384  1683079      c          banks  2023-12-29 08:00:00   
91385  1683204      c          banks  2023-12-29 08:00:00   
91386  1683205      c          banks  2023-12-29 08:00:00   
91387  1683206      c          banks  2023-12-29 08:00:00   
91388  1683207      c          banks  2023-12-29 08:00:00   

                                        translated_title  \
0      disputes over loan guarantees for the copper m...   
1      700 million dollars in net foreign assets for ...   
2      the reshuffle of the board البنك المركزى.. and... 

In [ ]:
gold_prices_target.head()

In [ ]:
gold_prices_target['Date'] = pd.to_datetime(gold_prices_target['date'])
gold_prices_target['Date'] = gold_prices_target['Date'].dt.date
gold_prices_target.drop(columns=['date'], inplace=True)
gold_prices_target.set_index('Date' , inplace=True)

In [ ]:
crude_oil_prices.head()

In [ ]:
crude_oil_prices['Date'] = pd.to_datetime(crude_oil_prices['Date'])
crude_oil_prices['Date'] = crude_oil_prices['Date'].dt.date
crude_oil_prices.set_index('Date' , inplace=True)

In [ ]:
features_df = pd.DataFrame(crude_oil_prices['Europe Brent Crude Oil (Dollars per Barrel)'].pct_change() * 100).shift(-1)
X = features_df[['Europe Brent Crude Oil (Dollars per Barrel)']]
X

# EDA

In [ ]:
gold_prices_target.describe()

In [ ]:
px.line(gold_prices_target)

In [ ]:
crude_oil_prices.describe()

In [ ]:
px.line(crude_oil_prices)

# Feature Selection

In [ ]:
features_df = pd.merge(gold_prices_target, (crude_oil_prices['Europe Brent Crude Oil (Dollars per Barrel)'].pct_change() * 100).shift(-1), on='Date', how='left')
features_df.fillna(0, inplace=True)
features_df

In [ ]:
px.line(features_df)

# Modelling

In [ ]:
X = features_df[['Europe Brent Crude Oil (Dollars per Barrel)']]  
y = features_df['pct_change']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Validation

In [ ]:
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
inference_time = end_time - start_time

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

mda = np.mean(np.sign(np.diff(y_test)) == np.sign(np.diff(y_pred)))

y_test_buckets = pd.qcut(y_test, 4, labels=False) 
y_pred_buckets = pd.qcut(y_pred, 4, labels=False)

f1 = f1_score(y_test_buckets, y_pred_buckets, average='weighted')

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Directional Accuracy: {mda}')
print(f'Bucketized F1 Score: {f1}')
print(f'Inference Time (seconds): {inference_time}')


## Visualize the results

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(features_df.index, y, label='Actual Target Variable', color='blue')
plt.plot(X_test.index, y_pred, label='Predicted Target Variable', color='red')
plt.title('Linear Regression Model')
plt.xlabel('Date')
plt.ylabel('Target Variable')
plt.legend()
plt.show()

# Saving Model

In [ ]:
with open('../Pickles/linear_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Loading Model

In [ ]:
with open('../Pickles/linear_regression_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

y_pred = loaded_model.predict(X_test)
output_df = pd.DataFrame({
    'date': X_test.index,
    'prediction': y_pred.flatten()
})
output_df